In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv('nwosu.csv',sep= ',')
df.head()

,Sex,Age (yrs),Age<5 Code,Age<3 Code,Age<10 Code,Ht(cm )at Dx,Ht SDS at Dx,Wt (kg) at Dx,Wt SDS at Dx,BMI (kg/m2)at Dx,...,# of Positive Antibodies,Antibodies Code,pH (7.35-7.45),PhCODE,HCO3 (23-28 mmol/L),ln(HCo3),HCO3 <15 Code,Ketonemia or ketonuria,DKA code,Polyuria&Polydipsia
0,1,10.74,0,0,0,152.9,1.608,58.1,2.213,24.80,...,3,2,7.43,0.0,26.2,3.27,0.0,0.0,0.0,1.0
1,0,6.32,0,0,1,119.9,0.700,23.8,0.833,16.53,...,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,14.07,0,0,0,156.4,-0.876,53.6,0.278,21.80,...,1,1,7.36,0.0,27.1,3.30,0.0,0.0,0.0,1.0
3,0,9.24,0,0,1,126.9,-1.084,24.5,-1.080,15.20,...,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,5.57,0,0,1,115.4,0.890,21.1,0.716,15.95,...,1,1,7.12,1.0,6.2,1.82,1.0,1.0,1.0,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 92 columns):
Sex                                 204 non-null int64
Age (yrs)                           204 non-null float64
Age<5 Code                          204 non-null int64
Age<3 Code                          204 non-null int64
Age<10 Code                         204 non-null int64
Ht(cm )at Dx                        204 non-null float64
Ht SDS at Dx                        200 non-null float64
Wt (kg) at Dx                       204 non-null float64
Wt SDS at Dx                        200 non-null float64
BMI (kg/m2)at Dx                    204 non-null float64
BMI SDS at Dx                       200 non-null float64
BMI% at Dx                          196 non-null float64
BMICode4                            196 non-null float64
BMICode2                            196 non-null float64
TDD LAI at Dx                       203 non-null float64
TDD SAI at Dx                       202 non-null

## Data Cleaning

In [5]:
target = 'PCROutcome'

### Replace empty or errors by np.nan

In [6]:
# replace empty/erros by np.nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?',np.nan,regex=True)
df = df.replace('\<',np.nan,regex=True)
#df

### Remove bad columns

In [7]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df  

In [8]:
df = remove_bad_columns(df,80)
df.info()

number of bad columns: 20
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 72 columns):
Sex                                 204 non-null int64
Age (yrs)                           204 non-null float64
Age<5 Code                          204 non-null int64
Age<3 Code                          204 non-null int64
Age<10 Code                         204 non-null int64
Ht(cm )at Dx                        204 non-null float64
Ht SDS at Dx                        200 non-null float64
Wt (kg) at Dx                       204 non-null float64
Wt SDS at Dx                        200 non-null float64
BMI (kg/m2)at Dx                    204 non-null float64
BMI SDS at Dx                       200 non-null float64
BMI% at Dx                          196 non-null float64
BMICode4                            196 non-null float64
BMICode2                            196 non-null float64
TDD LAI at Dx                       203 non-null float64
TDD SAI at Dx         

### Remove bad rows

In [9]:
 # Find rows where target is missing
def find_missing_target_rows(df,target):
    # find rows where target is missing
    missing_row = df[target].isnull()
    print('Number of rows where target are missing:')
    print(sum(missing_row))

    #df = df[~missing_row]
    missing_row_indices = np.array([t for t in range(df.shape[0]) if missing_row[t]])
    
    return missing_row_indices

In [10]:
missing_target_rows = find_missing_target_rows(df,target)

Number of rows where target are missing:
0


We find bad rows which contain too many missing values, then remove them.

In [11]:
def find_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    #df = df.drop(bad_row)
    #df.info()
    return bad_row

In [12]:
bad_rows = find_bad_rows(df,10)
df.info()

number of bad rows: 53
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 72 columns):
Sex                                 204 non-null int64
Age (yrs)                           204 non-null float64
Age<5 Code                          204 non-null int64
Age<3 Code                          204 non-null int64
Age<10 Code                         204 non-null int64
Ht(cm )at Dx                        204 non-null float64
Ht SDS at Dx                        200 non-null float64
Wt (kg) at Dx                       204 non-null float64
Wt SDS at Dx                        200 non-null float64
BMI (kg/m2)at Dx                    204 non-null float64
BMI SDS at Dx                       200 non-null float64
BMI% at Dx                          196 non-null float64
BMICode4                            196 non-null float64
BMICode2                            196 non-null float64
TDD LAI at Dx                       203 non-null float64
TDD SAI at Dx            

In [13]:
del_rows = np.union1d(missing_target_rows,bad_rows)
print('number of rows need to delete:',len(del_rows))

df = df.drop(del_rows)

number of rows need to delete: 53


### Separate target and attributes

In [14]:
dfx = df.drop(target,axis=1)
dfy = df[target]

### Find variable type

In [15]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[  2 144   2   2   2 138 146 125 146 105 144  74   4   2  31  32  63 136
  36  47  75 129   2  48  62  94 140  57  64  91  59  71  94 122   2   7
   6  11  61  43  50   2  43  41  37  42  43 132 141   2 138 126   2   2
   9   8  13   2  35  35   2   4   2  49   2 105  86   2   2   2   1]


In [16]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [17]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: [0, 2, 3, 4, 13, 22, 34, 41, 49, 52, 53, 57, 60, 62, 64, 67, 68, 69]
i_category: [12, 61, 70]
variable type: [1. 3. 1. 1. 1. 3. 3. 3. 3. 3. 3. 3. 2. 1. 3. 3. 3. 3. 3. 3. 3. 3. 1. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 1. 3. 3. 3. 3. 3. 3. 1. 3. 3. 3. 3. 3. 3.
 3. 1. 3. 3. 1. 1. 3. 3. 3. 1. 3. 3. 1. 2. 1. 3. 1. 3. 3. 1. 1. 1. 2.]


### Impute missing values of attributes

In [18]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [19]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 199
Data columns (total 71 columns):
Sex                                 151 non-null int64
Age (yrs)                           151 non-null float64
Age<5 Code                          151 non-null int64
Age<3 Code                          151 non-null int64
Age<10 Code                         151 non-null int64
Ht(cm )at Dx                        151 non-null float64
Ht SDS at Dx                        151 non-null float64
Wt (kg) at Dx                       151 non-null float64
Wt SDS at Dx                        151 non-null float64
BMI (kg/m2)at Dx                    151 non-null float64
BMI SDS at Dx                       151 non-null float64
BMI% at Dx                          151 non-null float64
BMICode4                            151 non-null float64
BMICode2                            151 non-null float64
TDD LAI at Dx                       151 non-null float64
TDD SAI at Dx                       151 non-null

## Data Processing

### Attributes

In [20]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [21]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(151, 77)
[[ 1.   10.74 -1.   ... -1.   -1.    1.  ]
 [-1.    9.24 -1.   ... -1.   -1.    1.  ]
 [-1.    5.57 -1.   ...  1.    1.    1.  ]
 ...
 [-1.    5.99 -1.   ...  1.    1.    1.  ]
 [-1.   11.15 -1.   ...  1.    1.    1.  ]
 [-1.    5.32 -1.   ... -1.   -1.    1.  ]]


### Target

In [22]:
y = np.array(dfy)
#print(np.unique(y,return_counts=True))

# convert taget to 0 and 1
y_new = y
#y_new = np.ones(y.shape[0])
#y_new[y =='No'] = 0

print(np.unique(y_new,return_counts=True))

(array([0, 1]), array([59, 92]))


In [23]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('nwosu_processed.dat',xy_new,fmt='%f')